# GRU Seed Expansion — pearson_v1 seeds 51-70 (Colab)
Trains 20 new pearson_v1 GRU seeds on Colab GPU.
Run the tightwd_v2 seeds (54-73) on Kaggle in parallel.

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

# Mount Drive for saving outputs
from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

# Install pinned kaggle + set credentials
!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "KGAT_17c43012d9e77edf2c183a25acb1489b"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Download + unzip dataset
os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup — clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
os.system(f"rm -rf {REPO}")
os.system(f"git clone https://github.com/vincentvdo6/competition_package.git {REPO}")
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

# Link data from Kaggle download
os.system('ln -sf /content/data/train.parquet datasets/train.parquet')
os.system('ln -sf /content/data/valid.parquet datasets/valid.parquet')

# Verify
assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
print("Commit:", subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip())
print(f"GPU: {os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()}")
print("Ready!")

In [ ]:
# Cell 2: pearson_v1 seeds 51-60 (10 seeds, ~16 min on T4)
# Existing p1: 42-50 (9 seeds). This adds 10 more.
import os
os.chdir("/content/competition_package")
for seed in range(51, 61):
    print(f"\n{'='*60}")
    print(f"Training gru_pearson_v1 seed {seed}")
    print(f"{'='*60}")
    os.system(
        f"python -u scripts/train.py "
        f"--config configs/gru_pearson_v1.yaml "
        f"--seed {seed} --device cuda"
    )
print("\nBatch 1 done: p1 seeds 51-60")

In [ ]:
# Cell 3: pearson_v1 seeds 61-70 (10 seeds, ~16 min on T4)
import os
os.chdir("/content/competition_package")
for seed in range(61, 71):
    print(f"\n{'='*60}")
    print(f"Training gru_pearson_v1 seed {seed}")
    print(f"{'='*60}")
    os.system(
        f"python -u scripts/train.py "
        f"--config configs/gru_pearson_v1.yaml "
        f"--seed {seed} --device cuda"
    )
print("\nBatch 2 done: p1 seeds 61-70")

In [ ]:
# Cell 4: Strip checkpoints + copy normalizers + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

for pt in sorted(glob.glob("logs/*.pt")):
    try:
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    except TypeError:
        ckpt = torch.load(pt, map_location="cpu")
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
    }
    out = f"logs/slim/{os.path.basename(pt)}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{os.path.basename(pt)}: {orig:.1f}MB -> {new:.1f}MB")

for npz in sorted(glob.glob("logs/normalizer_*.npz")):
    shutil.copy(npz, f"logs/slim/{os.path.basename(npz)}")
    print(f"Copied {os.path.basename(npz)}")

print(f"\n--- logs/slim/ contents ({len(os.listdir('logs/slim'))} files) ---")
for f in sorted(os.listdir("logs/slim")):
    sz = os.path.getsize(f"logs/slim/{f}") / 1e6
    print(f"  {f}: {sz:.1f}MB")

# Zip for download
shutil.make_archive("/content/p1_expansion", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/p1_expansion.zip") / 1e6
print(f"\np1_expansion.zip: {sz:.1f}MB")

# Save to Drive for persistence
shutil.copy("/content/p1_expansion.zip", "/content/drive/MyDrive/wunderfund/p1_expansion.zip")
print("Saved to Drive: MyDrive/wunderfund/p1_expansion.zip")

In [ ]:
# Cell 5: Print validation scores (sorted best to worst)
import os, glob, torch
os.chdir("/content/competition_package")

results = []
for pt in sorted(glob.glob("logs/*.pt")):
    try:
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    except TypeError:
        ckpt = torch.load(pt, map_location="cpu")
    score = ckpt.get("best_score", ckpt.get("val_score", None))
    name = os.path.basename(pt)
    if isinstance(score, (int, float)):
        results.append((name, float(score)))
    else:
        results.append((name, 0.0))

results.sort(key=lambda x: x[1], reverse=True)

print(f"{'Rank':<5} {'Model':<55} {'Val Score':>10}")
print("-" * 72)
for i, (name, score) in enumerate(results, 1):
    marker = " *" if score >= 0.264 else ""
    print(f"{i:<5} {name:<55} {score:>10.4f}{marker}")

scores = [s for _, s in results]
print(f"\n--- pearson_v1: {len(results)} seeds ---")
print(f"  Best 5: {sorted(scores, reverse=True)[:5]}")
print(f"  Mean: {sum(scores)/len(scores):.4f}")
print(f"  Std: {(sum((s-sum(scores)/len(scores))**2 for s in scores)/len(scores))**0.5:.4f}")